In [9]:
import os
import requests
import zipfile
import io
import pandas as pd
import chardet  # para detección automática de encoding

In [38]:
# pip install pandas requests openpyxl
# !pip install chardet
# !pip install unidecode

In [42]:
# Carpeta destino
os.makedirs("data/axa", exist_ok=True)

# Columnas oficiales
columnas = [
    "SINIESTRO","LATITUD","LONGITUD","CODIGO POSTAL","CALLE","COLONIA",
    "CAUSA SINIESTRO","TIPO VEHICULO","COLOR","MODELO","NIVEL DAÑO VEHICULO",
    "PUNTO DE IMPACTO","AÑO","MES","DÍA NUMERO","DIA","HORA","ESTADO","CIUDAD",
    "LESIONADOS","RELACION LESIONADOS","EDAD LESIONADO","GENERO LESIONADO",
    "NIVEL LESIONADO","HOSPITALIZADO","FALLECIDO","AMBULANCIA","ARBOL",
    "PIEDRA","DORMIDO","GRUA","OBRA CIVIL","PAVIMENTO MOJADO","EXPLOSION LLANTA",
    "VOLCADURA","PERDIDA TOTAL","CONDUCTOR DISTRAIDO","FUGA","ALCOHOL",
    "MOTOCICLETA","BICICLETA","SEGURO","TAXI","ANIMAL"
]

# Rango de años a descargar
anios = range(2018, 2025)

# Lista de DataFrames
dfs = []

for year in anios:
    url = f"https://files.i2ds.org/OpenDataAxaMx/incidentes_viales_{year}_axa.zip"
    print(f"Descargando {year} ...")

    try:
        # Descargar zip
        resp = requests.get(url)
        resp.raise_for_status()
        z = zipfile.ZipFile(io.BytesIO(resp.content))
        
        # Extraer CSV (usualmente solo hay uno)
        csv_name = [f for f in z.namelist() if f.endswith(".csv")][0]
        with z.open(csv_name) as f:
            # Leer el CSV
            if year < 2020:
                df = pd.read_csv(f, encoding="latin1", low_memory=False)
            else:
                # No tiene encabezados → los asignamos manualmente
                df = pd.read_csv(
                    f,
                    header=None,
                    names=columnas,
                    encoding="latin1",
                    low_memory=False
                )
            
            # Reemplazar valores nulos estándar
            df = df.replace({"\\N": pd.NA, " ": pd.NA, "": pd.NA})
            df["AÑO"] = year  # aseguramos el campo de año

            dfs.append(df)

    except Exception as e:
        print(f"⚠️ Error al procesar {year}: {e}")

# Concatenar todo
axa_all = pd.concat(dfs, ignore_index=True)

print(f"✅ Dataset combinado: {len(axa_all)} filas, {len(axa_all.columns)} columnas")
print("Ejemplo de columnas:", axa_all.columns[:10].tolist())

# Guardar limpio
axa_all.to_csv("data/axa/incidentes_viales_2015_2024.csv", index=False, encoding="utf-8")
print("💾 Guardado en data/axa/incidentes_viales_2015_2024.csv")


Descargando 2018 ...
Descargando 2019 ...
Descargando 2020 ...
Descargando 2021 ...
Descargando 2022 ...
Descargando 2023 ...
Descargando 2024 ...
✅ Dataset combinado: 927594 filas, 47 columnas
Ejemplo de columnas: ['SINIESTRO', 'LATITUD', 'LONGITUD', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO', 'TIPO VEHICULO', 'COLOR', 'MODELO']
💾 Guardado en data/axa/incidentes_viales_2015_2024.csv


In [43]:
axa_all.head()

,SINIESTRO,LATITUD,LONGITUD,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO,NIVEL DAÃO VEHICULO,PUNTO DE IMPACTO,AÃO,MES,DÃA NUMERO,DIA,HORA,ESTADO,CIUDAD,LESIONADOS,RELACION LESIONADOS,EDAD LESIONADO,GENERO LESIONADO,NIVEL LESIONADO,HOSPITALIZADO,FALLECIDO,AMBULANCIA,ARBOL,PIEDRA,DORMIDO,GRUA,OBRA CIVIL,PAVIMENTO MOJADO,EXPLOSION LLANTA,VOLCADURA,PERDIDA TOTAL,CONDUCTOR DISTRAIDO,FUGA,ALCOHOL,MOTOCICLETA,BICICLETA,SEGURO,TAXI,ANIMAL,AÑO,NIVEL DAÑO VEHICULO,DÍA NUMERO
0,4379071,16.8529794,-99.9008608,39300,Av. Costera Miguel AlemÃ¡n,Del PanteÃ³n,COLISION Y/O VUELCO,Auto,GRIS,2005,<NA>,NaN,2018.0,ENERO,1.0,LUNES,1,GUERRERO,ACAPULCO DE JUAREZ,0,<NA>,0,NaN,<NA>,NO,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
1,4378995,16.8521022,-99.7969736,39715,MEX 200,Leonardo Rodriguez Alcaine,COLISION Y/O VUELCO,Auto,AZUL CIELO,2009,Bajo,Trasero,2018.0,ENERO,1.0,LUNES,3,GUERRERO,ACAPULCO DE JUAREZ,0,<NA>,0,NaN,<NA>,NO,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
2,4379228,16.8593584,-99.8437132,39845,Monte Everest MZA 11 LOTE 1,Praderas de Costa Azul,COLISION Y/O VUELCO,Auto,BLANCO,2013,Bajo,Frontal,2018.0,ENERO,1.0,LUNES,10,GUERRERO,ACAPULCO DE JUAREZ,0,<NA>,0,NaN,<NA>,NO,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
3,4379138,17.6388618,-101.5504036,40895,Valentina,La Ropa,COLISION Y/O VUELCO,Auto,BLANCO,2011,Bajo,Frontal,2018.0,ENERO,1.0,LUNES,11,GUERRERO,ZIHUATANEJO DE AZUETA,0,<NA>,0,NaN,<NA>,NO,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
4,4379144,0,0,0,AJUSTADOR EN SITIO NOE RAMOS NAVARRETE,Sin dato,COLISION Y/O VUELCO,Auto,ROJO,2012,Bajo,Costado izq central,2018.0,ENERO,1.0,LUNES,11,GUERRERO,ZIHUATANEJO DE AZUETA,0,<NA>,0,NaN,<NA>,NO,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN


In [44]:
axa_all.shape

(927594, 47)

In [45]:
print(axa_all.columns.tolist())

['SINIESTRO', 'LATITUD', 'LONGITUD', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO', 'TIPO VEHICULO', 'COLOR', 'MODELO', 'NIVEL DAÃ\x91O VEHICULO', 'PUNTO DE IMPACTO', 'AÃ\x91O', 'MES', 'DÃ\x8dA NUMERO', 'DIA', 'HORA', 'ESTADO', 'CIUDAD', 'LESIONADOS', 'RELACION LESIONADOS', 'EDAD LESIONADO', 'GENERO LESIONADO', 'NIVEL LESIONADO', 'HOSPITALIZADO', 'FALLECIDO', 'AMBULANCIA', 'ARBOL', 'PIEDRA', 'DORMIDO', 'GRUA', 'OBRA CIVIL', 'PAVIMENTO MOJADO', 'EXPLOSION LLANTA', 'VOLCADURA', 'PERDIDA TOTAL', 'CONDUCTOR DISTRAIDO', 'FUGA', 'ALCOHOL', 'MOTOCICLETA', 'BICICLETA', 'SEGURO', 'TAXI', 'ANIMAL', 'AÑO', 'NIVEL DAÑO VEHICULO', 'DÍA NUMERO']


In [46]:
import unidecode

axa_all.columns = (
    axa_all.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("ñ", "n")
    .str.replace("ã±", "n")
    .str.replace("ã\x91", "n")
    .map(unidecode.unidecode)  # elimina acentos y símbolos raros
)

In [47]:
# Ver columnas duplicadas exactas
duplicadas = axa_all.columns[axa_all.columns.duplicated()].unique()
print("Columnas duplicadas:", duplicadas)

Columnas duplicadas: Index(['ano', 'nivel_dano_vehiculo'], dtype='object')


In [48]:
for col in duplicadas:
    # Combinar valores no nulos si existen múltiples columnas iguales
    same_cols = [c for c in axa_all.columns if c == col]
    if len(same_cols) > 1:
        axa_all[col] = axa_all[same_cols].bfill(axis=1).iloc[:, 0]  # elige el primer valor no nulo
        axa_all.drop(columns=same_cols[1:], inplace=True)


C:\Users\MasterHDXD\AppData\Local\Temp\ipykernel_3544\2743531339.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  axa_all[col] = axa_all[same_cols].bfill(axis=1).iloc[:, 0]  # elige el primer valor no nulo


In [49]:
axa_all = axa_all.loc[:, ~axa_all.columns.str.contains("unnamed")]

In [50]:
print(f"Total de columnas limpias: {len(axa_all.columns)}")
print(sorted(axa_all.columns))


Total de columnas limpias: 43
['alcohol', 'ambulancia', 'animal', 'arbol', 'bicicleta', 'calle', 'causa_siniestro', 'ciudad', 'codigo_postal', 'colonia', 'color', 'conductor_distraido', 'daa_numero', 'dia', 'dia_numero', 'dormido', 'edad_lesionado', 'estado', 'explosion_llanta', 'fallecido', 'fuga', 'genero_lesionado', 'grua', 'hora', 'hospitalizado', 'latitud', 'lesionados', 'longitud', 'mes', 'modelo', 'motocicleta', 'nivel_lesionado', 'obra_civil', 'pavimento_mojado', 'perdida_total', 'piedra', 'punto_de_impacto', 'relacion_lesionados', 'seguro', 'siniestro', 'taxi', 'tipo_vehiculo', 'volcadura']
